### Annotation of Emotions

In this notebook, the annotation process of the dataset will be automated. To do this, we send the data to OpenAI with GPT40-mini using the Few Shot method and receive it annotated based on the emotions

ref: https://social-media-lab.net/processing/classification.html

In [1]:
!pip install -q openai backoff gpt-cost-estimator

In [24]:
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
import backoff
import re
import ast
from gpt_cost_estimator import CostEstimator

In [4]:
df = pd.read_csv("label_annotation.csv")

In [5]:
df.head()

,id,tweet
0,132520,@tl_trevaskis @Bpage5 @CheriJacobus Which is e...
1,16909,@realDonaldTrump Sen. Jeff Merkley released a ...
2,67861,@movieguy82 @PaulCogan @lisalocicerogh Trump t...
3,22168,@pleasexplainMrI @MingGao26 When US and Britis...
4,124830,So Obama along with his team of lowlifes spied...


In [6]:
print(len(df))

1041


## Setup GPT

In [7]:
example_text = """
Example 1:
Tweet: "@newsmax @bennyjohnson Biden plans to tax your 401K. Believe it. The democrats are spending so much money they need new revenue sources. Wake up America. This is YOUR MONEY!!!!"
Valence: Negative
Arousal: High
Example 2:
Tweet: "It is profoundly wrong that the Walton family of Walmart owns more wealth than the bottom 130 million Americans."
Valence: Negative
Arousal: Low
Example 3:
Tweet: "Congrats to Ivy Taylor, first African American mayor of San Antonio."
Valence: Positive
Arousal: Low
"""

In [8]:
system_prompt = """
You are an advanced AI model for classification. Your task is to annotate tweets by identifying the emotional tone based on two dimensions: valence (positive, neutral, or negative) and arousal (low, medium, or high). The goal is to analyze how understanding emotions in tweets can help us to combat fake news. The coding helps to understand how emotions are mapped in tweets and what this means for the spread of fake news.

The posts should be classified into 2 x 3 categories. Multiple answers are not possible in the individual blocks. The theoretical basis of my work assumes that users pursue the following emotions in social media:

1. Valence (Emotional Tone):
Positive: The tweet conveys positive emotions like happiness, excitement, or hope.
Neutral: The tweet doesn’t express strong emotional content or is emotionally neutral.
Negative: The tweet expresses negative emotions like anger, fear, or sadness.
2. Arousal (Emotional Intensity):
High: The tweet conveys strong, intense emotions. These tweets may feel urgent, excited, or highly emotional.
Medium: The tweet shows moderate emotional intensity. The emotion is present but not overwhelming.
Low: The tweet conveys mild or subdued emotions, such as calmness or slight sadness.

General Approach to Borderline Cases:
Focus on the most prominent emotion when deciding between two categories.
If the tweet seems to fall between two levels of arousal (e.g., between medium and high), consider the context:
Does the tweet show urgency or emotional intensity? (If yes, lean towards high arousal).
Is the tone emotional but more measured? (If yes, lean towards medium arousal).

Here are some classification Examples:
{example_text}


During classification, you receive the text of the tweet. Your task is to decide which level of valence (positive, neutral, negative) and arousal (high, medium, low) is present in the post. Return the applicable categories as a list.
"""

In [20]:
system_prompt = system_prompt.replace('{example_text}',example_text)

In [9]:
prompt = """
Please categorise the following tweet in the social media for the Valence category as ‘negative’, ‘neutral’ or ‘positive’.
For the Arousal category as, ‘high’, ‘low’, ‘medium’
Your answer for each category MUST be one of [‘negative’, ‘neutral’, ‘positive’] for Valence [‘high’, ‘low’, ‘medium’] for arousal and should be in lower case.
Text: [TEXT]
"""

In [17]:
api_key_name = "NLP"
api_key = "" # INSERT KEY
# Initialize OpenAI using the key
client = OpenAI(
    api_key=api_key
)

@CostEstimator()
def query_openai(model, temperature, messages, mock=True, completion_tokens=10):
    return client.chat.completions.create(
                      model=model,
                      temperature=temperature,
                      messages=messages,
                      max_tokens=600)

# We define the run_request method to wrap it with the @backoff decorator
@backoff.on_exception(backoff.expo, (openai.RateLimitError, openai.APIError))
def run_request(system_prompt, user_prompt, model, mock):
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]

  return query_openai(
          model=model,
          temperature=0.0,
          messages=messages,
          mock=mock
        )


In [11]:
print(len(df))

1041


## Running the Request

In [10]:
from tqdm.auto import tqdm

#@markdown Do you want to mock the OpenAI request (dry run) to calculate the estimated price?
MOCK = True # @param {type: "boolean"}
#@markdown Do you want to reset the cost estimation when running the query?
RESET_COST = True # @param {type: "boolean"}
#@markdown What's the column name to save the results of the data extraction task to?
COLUMN = 'Sentiment' # @param {type: "string"}
#@markdown Do you want to run the request on a smaller sample of the whole data? (Useful for testing). Enter 0 to run on the whole dataset.
SAMPLE_SIZE = len(df) # @param {type: "number", min: 0}
id = 0
#@markdown Which model do you want to use?
MODEL = "gpt-3.5-turbo-1106"#"gpt-4o-mini-2024-07-18	" 


# Initializing the empty column
if COLUMN not in df.columns:
  df[COLUMN] = None

# Reset Estimates
CostEstimator.reset()
print("Reset Cost Estimation")

filtered_df = df.copy()

# Skip previously annotated rows
filtered_df = filtered_df[pd.isna(filtered_df[COLUMN])]

if SAMPLE_SIZE > 0:
  filtered_df = filtered_df.sample(SAMPLE_SIZE)

for index, row in tqdm(filtered_df.iterrows(), total=len(filtered_df)):
    try:
        
        p = prompt.replace('[TEXT]', row['tweet'])
        response = run_request(system_prompt, p, MODEL, MOCK)
        if not MOCK:
          # Extract the response content
          # Adjust the following line according to the structure of the response
          r = response.choices[0].message.content
          # Update the 'new_df' DataFrame
          df.at[index, COLUMN] = r
          if id%1000 == 0:
                df.to_csv(f"annotations/{id}.csv")
          id +=1

    except Exception as e:
        print(f"An error occurred: {e}")
        # Optionally, handle the error (e.g., by logging or by setting a default value)

print()
df.to_csv("GPT4omini.csv")

Reset Cost Estimation


  0%|          | 0/49986 [00:00<?, ?it/s]

Cost: $0.0006 | Total: $28.1226


### Process Annotations

In [31]:
reg_one_valence = r"(?<=valence: )[a-z]+"
reg_two_arousal = r"(?<=arousal: )[a-z]+"

def checkt_contain(text, category):

    try:
        categories = {
            'arousal':["low","medium","high"],
            'valence':["negative", "neutral","positive"]
        }

        for label in categories[category]:
            if label in text:
                return label
    except:
        print(text)
    

def format_str(text):
    arousal = ''
    valence = ''
    text = str(text).lower()
    if '[' in text:
        try:
            return ast.literal_eval(text)
        except:
            pattern = r'valence:\s*(\w+).*arousal:\s*(\w+)'
            match = re.search(pattern, text)
            if match:
                valence, arousal = match.groups()
                return [valence, arousal]
            else:
                standardized_string = text.replace('‘', "'").replace('’', "'")
                return ast.literal_eval(standardized_string)

    else:
        match_valence = re.search(reg_one_valence, text)
        match_arousal = re.search(reg_two_arousal, text)

        if match_valence:
            valence = match_valence.group(0)
        else:
            valence = checkt_contain(text, 'valence')
        if match_arousal:
            arousal = match_arousal.group(0)
        else:
            arousal = checkt_contain(text, 'arousal')

    return [valence, arousal]


In [33]:
valence_list = []
arousal_list = []
for data in df['Sentiment']:
    valence, arousal = format_str(data)
    valence_list.append(valence)
    arousal_list.append(arousal)
df['valence'] = valence_list
df['arousal'] = arousal_list


In [34]:
df[['id','tweet', 'valence', 'arousal']].to_csv("annotation.csv")

### GPT 4o mini

In [27]:
client = OpenAI(api_key=api_key)
def call_api(p):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": p}
        ]
    )
    return response.choices[0].message.content

In [28]:
from tqdm.auto import tqdm

COLUMN = 'GPT4mini'
df[COLUMN] = None
for index, row in tqdm(df.iterrows(), total=len(df)):
        p = prompt.replace('[TEXT]', row['tweet'])
        response = call_api(p)
        df.at[index, COLUMN] = response

  0%|          | 0/1041 [00:00<?, ?it/s]

In [ ]:
valence_list = []
arousal_list = []
for data in df['GPT4mini']:
    valence, arousal = format_str(data)
    valence_list.append(valence)
    arousal_list.append(arousal)
df['valence'] = valence_list
df['arousal'] = arousal_list

In [30]:
df.to_csv('GPT4omini_comparison.csv')